In [1]:
 current_dir = "gdrive/My Drive/Colab Notebooks/Thesis/gold_standard"


import os
import glob
from google.colab import drive
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import re
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    RidgeCV,
    LassoCV,
    ElasticNetCV,
)


# Mount Google Drive
drive.mount('/content/gdrive')



Mounted at /content/gdrive


In [3]:
os.chdir(current_dir)

In [4]:
import pandas as pd
copy = pd.read_pickle("pos_dataframe.pkl")

## Find Best Score: Model, Feature Combination,

##Train: handbook
##Validate: doc
##Test: cbc & article

In [5]:
keep  = ['ADJ', 'ADP', 'ADV', 'AUX', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'SCONJ', 'VERB', 'group', 'name', "score"]

In [6]:
df = copy[keep]
# Split the DataFrame into 'train' and 'validate' based on the 'group' column
train = df[df['group'] == 'handbook']
validate = df[df['group'] == 'doc']
# 'test' set will include all other values in the 'group' column
test = df[~df['group'].isin(['handbook', 'doc'])]


In [7]:
models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    SVR(),
]
features = ['ADJ', 'ADP', 'ADV', 'AUX', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'SCONJ', 'VERB',]

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneOut, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def create_and_evaluate_model(data, feature_columns, model):
    """
    Creates a text processing pipeline with the specified model, evaluates it using LOOCV,
    and fits the model to the entire dataset.

    Parameters:
    data (DataFrame): The dataset containing features and the target variable.
    feature_columns (list of str): List of column names to be used as features.
    model (BaseEstimator): The machine learning model to be used.

    Returns:
    dict: A dictionary containing the trained pipeline, model info, feature columns, and evaluation metrics (R², MSE, RMSE, MAE).
    """
    # Define a column transformer for text vectorization
    column_transformer = ColumnTransformer(
        [(col, CountVectorizer(), col) for col in feature_columns]
    )

    # Create a pipeline with the column transformer and the specified model
    pipeline = Pipeline([
        ('transformer', column_transformer),
        ('model', model)
    ])

    # Perform Leave-One-Out Cross-Validation
    loo = LeaveOneOut()
    y_pred = cross_val_predict(pipeline, data[feature_columns], data['score'], cv=loo)
    y_true = data['score']

    # Fit the pipeline to the entire dataset
    pipeline.fit(data[feature_columns], data['score'])

    # Calculate the evaluation metrics
    metrics = {
        'R²': r2_score(y_true, y_pred),
        'MSE': mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred)
    }

    # Return a dictionary containing the trained pipeline, model info, feature columns, and metrics
    return {
        'Trained Pipeline': pipeline,
        'Model': str(model),
        'Feature Columns': feature_columns,
        'Evaluation Metrics': metrics
    }






In [9]:
from itertools import combinations

features = ['ADJ', 'ADP', 'ADV', 'AUX', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'SCONJ', 'VERB']
feature_combos = [list(combo) for combo in combinations(features, 2)]


In [10]:
pipelines = []

for model in models:
  for feature_columns in feature_combos:
    result = create_and_evaluate_model(train, feature_columns, model)
    if result["Evaluation Metrics"]["R²"] > 0.9:
      print(result["Model"], result["Feature Columns"], result["Evaluation Metrics"])
      pipelines.append(result)








Lasso() ['ADV', 'AUX'] {'R²': 0.9268077147642332, 'MSE': 1.797602525390434, 'RMSE': 1.340747002752732, 'MAE': 1.0423391458794256}
ElasticNet() ['ADV', 'AUX'] {'R²': 0.9710306869781741, 'MSE': 0.711486327816045, 'RMSE': 0.8434964895102084, 'MAE': 0.7450433828128127}


In [190]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def validate_model(data, pipeline, feature_columns):
    """
    Validates a trained model pipeline on a given dataset and returns evaluation results.

    Parameters:
    data (DataFrame): The dataset for validation, containing features, target variable 'score', and 'name'.
    pipeline (Pipeline): The trained model pipeline.
    feature_columns (list of str): List of column names to be used as features.

    Returns:
    dict: A dictionary containing the names, actual and predicted values, and evaluation metrics.
    """
    # Predict using the pipeline
    y_pred = pipeline.predict(data[feature_columns])
    y_true = data['score']

    # Calculate evaluation metrics
    metrics = {
        'R²': r2_score(y_true, y_pred),
        'MSE': mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred)
    }

    # Prepare the results
    results = {
        'Names': data['name'].tolist(),
        'Actual Scores': y_true.tolist(),
        'Predicted Scores': y_pred.tolist(),
        'Evaluation Metrics': metrics
    }

    return results




NameError: ignored

In [221]:
pipelines[0]

{'Trained Pipeline': Pipeline(steps=[('transformer',
                  ColumnTransformer(transformers=[('ADV', CountVectorizer(),
                                                   'ADV'),
                                                  ('AUX', CountVectorizer(),
                                                   'AUX')])),
                 ('model', Lasso())]),
 'Model': 'Lasso()',
 'Feature Columns': ['ADV', 'AUX'],
 'Evaluation Metrics': {'R²': 0.9268077147642332,
  'MSE': 1.797602525390434,
  'RMSE': 1.340747002752732,
  'MAE': 1.0423391458794256}}

In [209]:
best_pipe = pipelines[1]['Trained Pipeline']

ValueError: ignored

AUX  \
0  was was s had would was do be is m are can be ...   
1                              was was be is is m be   

                                                 ADV  
0  very together away ever so just so right sure ...  
1                        then someday very at around

In [215]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_predict, LeaveOneOut
import numpy as np

def create_and_evaluate_model(data, feature_columns, model):
    # Define a column transformer for text vectorization
    column_transformer = ColumnTransformer(
        [(col, CountVectorizer(), col) for col in feature_columns]
    )

    # Create a pipeline with the column transformer and the specified model
    pipeline = Pipeline([
        ('transformer', column_transformer),
        ('model', model)
    ])

    # Fit the pipeline to the entire dataset
    pipeline.fit(data[feature_columns], data['score'])

    # Perform Leave-One-Out Cross-Validation
    loo = LeaveOneOut()
    y_pred = cross_val_predict(pipeline, data[feature_columns], data['score'], cv=loo)
    y_true = data['score']

    # Calculate the evaluation metrics
    metrics = {
        'R²': r2_score(y_true, y_pred),
        'MSE': mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred)
    }

    # Return a dictionary containing the trained pipeline, model info, feature columns, and metrics
    return {
        'Trained Pipeline': pipeline,
        'Model': str(model),
        'Feature Columns': feature_columns,
        'Evaluation Metrics': metrics
    }


In [227]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def validate_model(data, pipeline):
    """
    Validates a trained model pipeline on a given dataset and returns evaluation results along with predictions.

    Parameters:
    data (DataFrame): The dataset for validation, containing features, target variable 'score', and 'name'.
    pipeline (Pipeline): The trained model pipeline.

    Returns:
    dict: A dictionary containing the names, actual and predicted values, and evaluation metrics.
    """
    # Predict using the pipeline
    y_pred = pipeline.predict(data.drop(columns=['score', 'name'], errors='ignore'))
    y_true = data['score']

    # Calculate evaluation metrics
    metrics = {
        'R²': r2_score(y_true, y_pred),
        'MSE': mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred)
    }

    # Print Names, Actual Scores, and Predicted Scores
    for name, actual, predicted in zip(data['name'], y_true, y_pred):
        print(f"Name: {name}, Actual Score: {actual}, Predicted Score: {predicted}")

    return metrics




In [231]:
my_pipe = create_and_evaluate_model(train, ["AUX", "ADV", "PRON"], ElasticNet())["Trained Pipeline"]

In [234]:
# Example usage
# Assuming 'validate_data' is your validation dataset and 'trained_pipeline' is your trained pipeline
validation_results = validate_model(validate, my_pipe)
print(validation_results)


Name: wayne, Actual Score: 40.0, Predicted Score: 38.96395284294332
Name: carl, Actual Score: 36.0, Predicted Score: 39.59816618999853
{'R²': -0.7525242053117203, 'MSE': 7.010096821246881, 'RMSE': 2.647658743351734, 'MAE': 2.3171066735276042}
